In [ ]:
from flask import Flask, request, jsonify, send_file, make_response
from werkzeug.exceptions import RequestURITooLarge
from werkzeug.wrappers import Response
import youtube_dl
import os
import json
import requests
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/slashing', methods=['GET'])
def slashing_api():
    url = request.args.get('url')
    
    # delete source.mp3 file
    os.system('del source.mp3')
    
    mp3_file_name = get_mp3(url)
    
    # extract vocal from source.mp3 file
    os.system('echo spleeter_starts')
    os.system('python -m spleeter separate -i source.mp3 -p spleeter:4stems -o output')
    
    # delete source.mp3 file
    # os.system('del source.mp3')
    
    # for debug
    print('spleeter finished')
    
    # send vocals.wav to deepspeech server
    url = 'http://54.152.57.69/receive.php'
    files = {'uploadFile': open('./output/source/vocals.wav', 'rb')}
    res = requests.post(url, files = files)
    
    return str(res.text)

def get_mp3(url):
    video_info = youtube_dl.YoutubeDL().extract_info(
        url, download=False
    )
    file_name = "source.mp3"
    options = {
        'format' : 'bestaudio/best',
        'keepvideo' : False,
        'outtmpl' : file_name,
        'postprocessors' : [{
            'key' : 'FFmpegExtractAudio',
            'preferredcodec' : 'mp3',
            'preferredquality' : '192',
        }]
    }
    
    with youtube_dl.YoutubeDL(options) as download:
        download.download([url])
        
    return file_name

if __name__ == '__main__':
    app.run(host = '0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


[youtube] jO2viLEW-1A: Downloading webpage
[youtube] jO2viLEW-1A: Downloading webpage
[download] Destination: source.mp3
[download] 100% of 2.83MiB in 00:00                          
[ffmpeg] Post-process file source.mp3 exists, skipping
spleeter finished


182.211.14.72 - - [17/May/2021 15:59:28] "GET /slashing?url=https://www.youtube.com/watch?v=jO2viLEW-1A HTTP/1.1" 200 -
